In [4]:
import ast
import pandas as pd

# ------------------------------------------------------------------
# Config
# ------------------------------------------------------------------
INPUT_CSV = "pde_sum_all_files.csv"
OUTPUT_CSV = "pde_sum_all_files_with_names_newlight.csv"

# ------------------------------------------------------------------
# Mapping: (ADC, local channels) -> trap/tile name
# Names built from "Light Tile Type" + "Light Tile #"
# ------------------------------------------------------------------
TRAP_NAME_MAP = [
    # ----- ADC 0 (ACLs module 0, made-up names) -----
    dict(adc=0, channels=[4,5,6,7,8,9], name="ACL 2.01"),           # -z, bottom, tpc 0
    dict(adc=0, channels=[10,11,12,13,14,15], name="ACL 2.02"),     # -z, top, tpc 0
    dict(adc=0, channels=[20,21,22,23,24,25], name="ACL 2.03")      # +z bottom, tpc 0
    dict(adc=0, channels=[26,27,28,29,30,31], name="ACL 2.04"),     # +z top, tpc 0
    dict(adc=0, channels=[52,53,54,55,56,57], name="ACL 2.05"),     # -z bottom, tpc 1
    dict(adc=0, channels=[58,59,60,61,62,63], name="ACL 2.06"),     # -z top, tpc 1
    dict(adc=0, channels=[36,37,38,39,40,41], name="ACL 2.07"),     # +z bottom, tpc 1
    dict(adc=0, channels=[42,43,44,45,46,47], name="ACL 2.08"),     # +z top, tpc 1
    
    # ----- ADC 1 (LCMs module 0, made-up names) -----
    dict(adc=1, channels=[4,5,6,7,8,9], name="LCM 100,101,102"),       # -z, bottom, tpc 0
    dict(adc=1, channels=[10,11,12,13,14,15], name="LCM 103,104,105"), # -z, top, tpc 0
    dict(adc=1, channels=[20,21,22,23,24,25], name="LCM 106,107,108"), # +z bottom, tpc 0
    dict(adc=1, channels=[26,27,28,29,30,31], name="LCM 109,110,111"), # +z top, tpc 0
    dict(adc=1, channels=[52,53,54,55,56,57], name="LCM 112,113,114"), # -z bottom, tpc 1
    dict(adc=1, channels=[58,59,60,61,62,63], name="LCM 115,116,117"), # -z top, tpc 1
    dict(adc=1, channels=[36,37,38,39,40,41], name="LCM 118,119,120"), # +z bottom, tpc 1
    dict(adc=1, channels=[42,43,44,45,46,47], name="LCM 121,122,123"), # +z top, tpc 1
    
    # ----- ADC 2 (ACLs module 1, made-up names) -----
    dict(adc=2, channels=[4,5,6,7,8,9], name="ACL 2.09"),           # -z, bottom, tpc 2
    dict(adc=2, channels=[10,11,12,13,14,15], name="ACL 2.10"),     # -z, top, tpc 2
    dict(adc=2, channels=[20,21,22,23,24,25], name="ACL 2.11"),     # +z bottom, tpc 2
    dict(adc=2, channels=[26,27,28,29,30,31], name="ACL 2.12"),     # +z top, tpc 2
    dict(adc=2, channels=[52,53,54,55,56,57], name="ACL 2.13"),     # -z bottom, tpc 3
    dict(adc=2, channels=[58,59,60,61,62,63], name="ACL 2.14"),     # -z top, tpc 3
    dict(adc=2, channels=[36,37,38,39,40,41], name="ACL 2.15"),     # +z bottom, tpc 3
    dict(adc=2, channels=[42,43,44,45,46,47], name="ACL 2.16"),     # +z top, tpc 3

    # ----- ADC 3 (LCMs module 1, made-up names) -----
    dict(adc=3, channels=[20,21,22,23,24,25], name="LCM 124,125,126"),    # -z, bottom, tpc 2
    dict(adc=3, channels=[26,27,28,29,30,31], name="LCM 127,128,129"),    # -z, top, tpc 2
    dict(adc=3, channels=[4,5,6,7,8,9], name="LCM 130,131,132"),          # +z bottom, tpc 2
    dict(adc=3, channels=[10,11,12,13,14,15], name="LCM 133,134,135"),    # +z top, tpc 2
    dict(adc=3, channels=[52,53,54,55,56,57], name="LCM 136,137,138"),    # -z bottom, tpc 3
    dict(adc=3, channels=[58,59,60,61,62,63], name="LCM 139,140,141"),    # -z top, tpc 3
    dict(adc=3, channels=[36,37,38,39,40,41], name="LCM 142,143,144"),    # +z bottom, tpc 3
    dict(adc=3, channels=[42,43,44,45,46,47], name="LCM 145,146,147"),    # +z top, tpc 3

    # ----- ADC 4 (ACLs module 2, made-up names) -----
    dict(adc=4, channels=[42,43,44,45,46,47], name="ACL 2.17"),           # -z, bottom, tpc 4
    dict(adc=4, channels=[36,37,38,39,40,41], name="ACL 2.18"),           # -z, top, tpc 4
    dict(adc=4, channels=[52,53,54,55,56,57], name="LCM 148,149,150"),    # -z top, tpc 4
    dict(adc=4, channels=[58,59,60,61,62,63], name="LCM 151,152,153"),    # +z bottom, tpc 4
    dict(adc=4, channels=[26,27,28,29,30,31], name="ACL 2.19"),           # -z bottom, tpc 5
    dict(adc=4, channels=[20,21,22,23,24,25], name="ACL 2.20"),           # -z top, tpc 5
    dict(adc=4, channels=[10,11,12,13,14,15], name="ACL 2.21"),           # +z bottom, tpc 5
    dict(adc=4, channels=[4,5,6,7,8,9], name="ACL 2.22"),                 # +z top, tpc 5

    # ----- ADC 5 (LCMs module 2, made-up names) -----
    dict(adc=5, channels=[42,43,44,45,46,47], name="LCM 154,155,156"),    # -z, bottom, tpc 4
    dict(adc=5, channels=[58,59,60,61,62,63], name="ACL 2.23"),           # +z, bottom, tpc 4
    dict(adc=5, channels=[52,53,54,55,56,57], name="ACL 2.24"),           # +z top, tpc 4
    dict(adc=5, channels=[36,37,38,39,40,41], name="LCM 157,158,159"),    # +z top, tpc 4
    dict(adc=5, channels=[26,27,28,29,30,31], name="LCM 160,161,162"),    # -z bottom, tpc 5
    dict(adc=5, channels=[20,21,22,23,24,25], name="LCM 163,164,165"),    # -z top, tpc 5
    dict(adc=5, channels=[10,11,12,13,14,15], name="LCM 166,167,168"),    # +z bottom, tpc 5
    dict(adc=5, channels=[4,5,6,7,8,9], name="LCM 169,170,171"),          # +z top, tpc 5

    # ----- ADC 6 (ACLs module 3, made-up names) -----
    dict(adc=6, channels=[4,5,6,7,8,9], name="ACL 2.25"),           # -z, bottom, tpc 6
    dict(adc=6, channels=[10,11,12,13,14,15], name="ACL 2.26"),     # -z, top, tpc 6
    dict(adc=6, channels=[20,21,22,23,24,25], name="ACL 2.27"),     # +z bottom, tpc 6
    dict(adc=6, channels=[26,27,28,29,30,31], name="ACL 2.28"),     # +z top, tpc 6
    dict(adc=6, channels=[52,53,54,55,56,57], name="ACL 2.29"),     # -z bottom, tpc 7
    dict(adc=6, channels=[58,59,60,61,62,63], name="ACL 2.30"),     # -z top, tpc 7
    dict(adc=6, channels=[36,37,38,39,40,41], name="ACL 2.31"),     # +z bottom, tpc 7
    dict(adc=6, channels=[42,43,44,45,46,47], name="ACL 2.32"),     # +z top, tpc 7

    # ----- ADC 7 (LCMs module 3, made-up names) -----
    dict(adc=7, channels=[4,5,6,7,8,9], name="LCM 172,173,174"),       # -z, bottom, tpc 6
    dict(adc=7, channels=[10,11,12,13,14,15], name="LCM 175,176,177"), # -z, top, tpc 6
    dict(adc=7, channels=[20,21,22,23,24,25], name="LCM 178,179,180"), # +z bottom, tpc 6
    dict(adc=7, channels=[26,27,28,29,30,31], name="LCM 181,182,183"), # +z top, tpc 6
    dict(adc=7, channels=[52,53,54,55,56,57], name="LCM 184,185,186"), # -z bottom, tpc 7
    dict(adc=7, channels=[58,59,60,61,62,63], name="LCM 187,188,189"), # -z top, tpc 7
    dict(adc=7, channels=[36,37,38,39,40,41], name="LCM 190,191,192"), # +z bottom, tpc 7
    dict(adc=7, channels=[42,43,44,45,46,47], name="LCM 193,194,195"), # +z top, tpc 7
]

# ------------------------------------------------------------------
# Helpers
# ------------------------------------------------------------------
def parse_ch_list(value):
    """
    Turn the ch_list column into a Python list of ints.
    Handles things like:
      "[0, 4]"       -> [0, 4]
      "0,4"          -> [0, 4]
      already-a-list -> itself
    """
    if isinstance(value, list):
        return value

    if pd.isna(value):
        return []

    if isinstance(value, str):
        text = value.strip()
        # case: string representation of a Python list
        if text.startswith("[") and text.endswith("]"):
            try:
                parsed = ast.literal_eval(text)
                if isinstance(parsed, (list, tuple)):
                    return [int(x) for x in parsed]
            except Exception:
                pass

        # fallback: comma-separated
        parts = [p.strip() for p in text.split(",") if p.strip()]
        out = []
        for p in parts:
            try:
                out.append(int(p))
            except ValueError:
                # ignore non-numeric garbage if any
                pass
        return out

    # anything else -> empty
    return []


def find_trap_name(adc_value, ch_list):
    """
    Find trap name by:
      - matching adc
      - checking if ANY channel in ch_list is in the mapping's channels
    If nothing matches, return "".
    """
    if ch_list is None:
        ch_list = []

    try:
        adc_int = int(adc_value)
    except Exception:
        return ""

    for entry in TRAP_NAME_MAP:
        if entry["adc"] != adc_int:
            continue
        channels = entry["channels"]
        # check intersection: any ch in row's ch_list that belongs to this trap
        if any((ch in channels) for ch in ch_list):
            return entry["name"]

    return ""


# ------------------------------------------------------------------
# Main
# ------------------------------------------------------------------
def main():
    # Read original CSV
    df = pd.read_csv(INPUT_CSV)

    # Parse ch_list into actual lists of ints
    df["ch_list_parsed"] = df["ch_list"].apply(parse_ch_list)

    # Compute trap_name for each row
    df["trap_name"] = [
        find_trap_name(row["adc"], row["ch_list_parsed"])
        for _, row in df.iterrows()
    ]

    # Drop helper column
    df = df.drop(columns=["ch_list_parsed"])

    # Write new CSV with extra column
    df.to_csv(OUTPUT_CSV, index=False)


if __name__ == "__main__":
    main()
